---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [13]:
movie_choice_df = pd.read_csv("Employee_Movie_Choices.txt", sep = "\t", header = None)
movie_choice_df.columns = ["Employee", "Movie"]
movie_choice_df.drop(0, axis = 0, inplace = True)
movie_choice_df.reset_index(inplace = True)
movie_choice_df = movie_choice_df.loc[:,['Employee', 'Movie']]
display(movie_choice_df)

,Employee,Movie
0,Andy,Anaconda
1,Andy,Mean Girls
2,Andy,The Matrix
3,Claude,Anaconda
4,Claude,Monty Python and the Holy Grail
5,Claude,Snakes on a Plane
6,Frida,The Matrix
7,Frida,The Shawshank Redemption
8,Frida,The Social Network
9,Georgia,Anaconda


In [14]:
def answer_one():
    G = nx.Graph()
    # Your Code Here
    for i in range(len(movie_choice_df)):
        G.add_edge(movie_choice_df.loc[i, "Employee"], movie_choice_df.loc[i, "Movie"])
    
    return G       # Your Answer Here

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [19]:
def answer_two():
    
    # Your Code Here
    G = answer_one()
    for node in G.nodes():
        if node in employees:
            G.add_node(node, type="employee")
        elif node in movies:
            G.add_node(node, type="movie")
    
    return G      # Your Answer Here

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [20]:
def answer_three():
        
    # Your Code Here
    G = answer_two()
    weighted_projected_graph = bipartite.weighted_projected_graph(G, employees)
    
    return weighted_projected_graph     # Your Answer Here

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [17]:
employee_relationship_df = pd.read_csv("Employee_Relationships.txt", sep = "\t", header = None)
employee_relationship_df.columns = ["Employee_1", "Employee_2", "Relationship_Score"]
display(employee_relationship_df)

,Employee_1,Employee_2,Relationship_Score
0,Andy,Claude,0
1,Andy,Frida,20
2,Andy,Georgia,-10
3,Andy,Joan,30
4,Andy,Lee,-10
5,Andy,Pablo,-10
6,Andy,Vincent,20
7,Claude,Frida,0
8,Claude,Georgia,90
9,Claude,Joan,0


In [21]:
G = answer_three()
G_df = pd.DataFrame(G.edges(data=True), columns=['Employee_1', 'Employee_2', 'Movie_Score'])
display(G_df)

,Employee_1,Employee_2,Movie_Score
0,Pablo,Frida,{'weight': 2}
1,Pablo,Andy,{'weight': 1}
2,Pablo,Vincent,{'weight': 1}
3,Lee,Joan,{'weight': 3}
4,Lee,Andy,{'weight': 1}
5,Frida,Andy,{'weight': 1}
6,Frida,Vincent,{'weight': 2}
7,Claude,Georgia,{'weight': 3}
8,Claude,Andy,{'weight': 1}
9,Joan,Andy,{'weight': 1}


In [22]:
def answer_four():
        
    # Your Code Here
    G = answer_three()
    Rel = nx.read_edgelist('Employee_Relationships.txt' ,data=[('relationship_score', int)])
    
    G_df = pd.DataFrame(G.edges(data=True), columns=['From', 'To', 'movies_score'])
    Rel_df = pd.DataFrame(Rel.edges(data=True), columns=['From', 'To', 'relationship_score'])
    
    G_copy_df = G_df.copy()
    G_copy_df.rename(columns={"From":"From1", "To":"From"}, inplace=True)
    G_copy_df.rename(columns={"From1":"To"}, inplace=True)
    
    G_final_df = pd.concat([G_df, G_copy_df])
    
    final_df = pd.merge(G_final_df, Rel_df, on = ['From', 'To'], how='right')
    
    final_df['movies_score'] = final_df['movies_score'].map(set_value)
    
    final_df['movies_score'] = final_df['movies_score'].map(lambda x: x['weight'])
    final_df['relationship_score'] = final_df['relationship_score'].map(lambda x: x['relationship_score'])
    
    value = final_df['movies_score'].corr(final_df['relationship_score'])
    
    return value

def set_value(val):
    if val is np.nan:
        return {'weight': 0}
    else:
        return val

In [23]:
answer_four()

0.78839622217334715